In [1]:
import pandas as pd
import numpy as np
import math
import datetime
import bandit.main as bd

In [2]:
def get_frame(filename):
    df = pd.read_excel(io=filename)
    # df_combined = pd.merge(df,df_lookup,on='Difficulty')
    # df_combined['Duration'] = df_combined['Time'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
    # df_combined['MatRate'] = df_combined['Mats'] / df_combined['Duration']
    # df_combined['DiffClass'] = df_combined['Class'] + '-' + df_combined['Difficulty']
    return df

In [3]:
import math
def get_ucb(row):
    cnt = max(row['count'],1)
    row_total = max(row['total'],1)
    div1 = 2 * math.log(row_total) / cnt
    min1 = min(0.25, row['var'] + div1)
    div2 = math.log(row_total) / cnt
    ucb = math.sqrt(div2 * min1)
    return ucb + row['mean']

In [4]:
# todo: combine bandit_class and bandit_diff
def df_bandit_class(df, classname):
    dfgb = df.groupby([classname])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(get_ucb, axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [5]:
def df_bandit_diff(df):
    dfgb = df.groupby(['goal'])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(get_ucb, axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [6]:
df = get_frame('GoalSetter.xlsx')

In [7]:
df = bd.reduce(df,'goal')

0 1.0 0


In [8]:
df_quality = bd.get_quality(df['score'])

In [9]:
df = df.join(df_quality).sort_values('Quality', ascending=False)

In [10]:
#df.loc[df.score.isnull(), 'Quality'] = 0.5

In [11]:
df.tail(10)

,date,goal,cohort,complete,score,Quality
5,2018-08-01,6,0,NaN,NaN,NaN
10,2018-08-16,11,1,NaN,NaN,NaN
11,2018-08-01,12,1,NaN,NaN,NaN
12,2018-08-01,13,1,NaN,NaN,NaN
14,2018-08-01,15,2,NaN,NaN,NaN
15,2018-08-01,16,2,NaN,NaN,NaN
16,2018-08-01,17,2,NaN,NaN,NaN
17,2018-08-01,18,2,NaN,NaN,NaN
18,2018-08-01,19,2,NaN,NaN,NaN
19,2018-08-01,20,2,NaN,NaN,NaN


In [12]:
df = df.sort_values('Quality', ascending=False)
df.head(10)

,date,goal,cohort,complete,score,Quality
9,2018-08-15,10,1,14.0,10.0,1.0
8,2018-08-14,9,1,10.0,9.0,0.9
6,2018-08-01,7,1,7.0,7.0,0.7
0,2018-08-10,1,0,1.0,1.0,0.1
7,2018-08-14,8,1,7.0,0.0,0.0
13,2018-08-01,14,2,4.0,0.0,0.0
1,2018-08-01,2,0,NaN,NaN,NaN
2,2018-08-01,3,0,NaN,NaN,NaN
3,2018-08-01,4,0,NaN,NaN,NaN
4,2018-08-01,5,0,NaN,NaN,NaN


In [13]:
df_class = df_bandit_class(df,'cohort')

In [14]:
df_class

,cohort,count,mean,var,total,donext
1,1,4,0.65,0.203333,6,0.984642
0,0,1,0.10,NaN,6,0.769283
2,2,1,0.00,NaN,6,0.669283


In [15]:
nextClass = df_class.iloc[0,0]

In [16]:
if df['complete'].isnull().sum() > 0:
    df_filtered = df[df['cohort'] == nextClass]
else:
    df_filtered = df

In [17]:
df_bandit_diff(df_filtered)

,goal,count,mean,var,total,donext
4,11,0,NaN,NaN,4,999.000000
5,12,0,NaN,NaN,4,999.000000
6,13,0,NaN,NaN,4,999.000000
3,10,1,1.0,NaN,4,1.588705
2,9,1,0.9,NaN,4,1.488705
0,7,1,0.7,NaN,4,1.288705
1,8,1,0.0,NaN,4,0.588705
